In [5]:
import boto3
import json
import pandas as pd
import configparser

In [6]:
config = configparser.ConfigParser()
config.read_file(open('capstone.cfg'))

In [7]:
iam_client = boto3.client('iam',
                       region_name=config.get('AWS','REGION'),
                       aws_access_key_id=config.get('AWS','AWS_ACCESS_KEY_ID'),
                       aws_secret_access_key=config.get('AWS','AWS_SECRET_ACCESS_KEY')
                      )

redshift_client = boto3.client('redshift',
                        region_name=config.get('AWS','REGION'),
                        aws_access_key_id=config.get('AWS','AWS_ACCESS_KEY_ID'),
                        aws_secret_access_key=config.get('AWS','AWS_SECRET_ACCESS_KEY')
                       )

ec2_client = boto3.resource('ec2',
                   region_name=config.get('AWS','REGION'),
                   aws_access_key_id=config.get('AWS','AWS_ACCESS_KEY_ID'),
                   aws_secret_access_key=config.get('AWS','AWS_SECRET_ACCESS_KEY')
                  )


In [10]:
try:
    print('===== Creating a new IAM Role =====')
    iam_client.create_role(
        Path='/',
        RoleName=config.get('IAM_ROLE','IAM_ROLE_NAME'),
        Description = "Allows Redshift clusters to call AWS services on your behalf.",
        AssumeRolePolicyDocument=json.dumps(
            {
                'Statement': [{
                    'Action': 'sts:AssumeRole',
                    'Effect': 'Allow',
                    'Principal': {
                        'Service': 'redshift.amazonaws.com'
                    }
                }],
                'Version': '2012-10-17'
            }
        )
    )

    print('===== Attaching AmazonS3ReadOnlyAccess policy with the IAM Role =====')
    iam_client.attach_role_policy(
        RoleName=config.get('IAM_ROLE','IAM_ROLE_NAME'),
        PolicyArn="arn:aws:iam::aws:policy/AmazonS3ReadOnlyAccess"
    )

except Exception as e:
    print(e)

===== Creating a new IAM Role =====
===== Attaching AmazonS3ReadOnlyAccess policy with the IAM Role =====


In [12]:
roleArn = iam_client.get_role(RoleName=config.get('IAM_ROLE','IAM_ROLE_NAME'))['Role']['Arn']

try:
    response = redshift_client.create_cluster(        
        #HW
        ClusterType=config.get('DWH','CLUSTER_TYPE'),
        NodeType=config.get('DWH','NODE_TYPE'),
        NumberOfNodes=int(config.get('DWH','NUM_NODES')),

        #Identifiers & Credentials
        DBName=config.get('CLUSTER','DB_NAME'),
        ClusterIdentifier=config.get('CLUSTER','CLUSTER_IDENTIFIER'),
        MasterUsername=config.get('CLUSTER','DB_USER'),
        MasterUserPassword=config.get('CLUSTER','DB_PASSWORD'),

        #Roles (for s3 access)
        IamRoles=[roleArn]  
    )
    print("==== Creating Redshift Cluster =====")
except Exception as e:
    print(e)

==== Creating Redshift Cluster =====


In [9]:
def pretty_redshift_props(props):
    pd.set_option('display.max_colwidth', -1)
    keys_to_show = ["ClusterIdentifier", "NodeType", "ClusterStatus", "MasterUsername", "DBName", "Endpoint", "NumberOfNodes", 'VpcId']
    x = [(k, v) for k,v in props.items() if k in keys_to_show]
    return pd.DataFrame(data=x, columns=["Key", "Value"])

In [10]:
def redshift_cluster_status(redshift_client, verbose=False):
    if not redshift_client:
        return
    
    cluster_props = redshift_client.describe_clusters(ClusterIdentifier=config.get('CLUSTER','CLUSTER_IDENTIFIER'))['Clusters'][0]
    if verbose:
        print(pretty_redshift_props(cluster_props))
    return cluster_props

In [18]:
roleArn

'arn:aws:iam::308990863356:role/capstoneRole'

In [19]:
def open_access_to_cluster(ec2_client, redshift_client):
    cluster_props = redshift_cluster_status(redshift_client)
    if not cluster_props:
        return
    
    try:
        vpc = ec2_client.Vpc(id=cluster_props['VpcId'])
        defaultSg = list(vpc.security_groups.all())[0]
        print(defaultSg)
        port = int(config.get('CLUSTER','DB_PORT'))
        defaultSg.authorize_ingress(
            GroupName=defaultSg.group_name,
            CidrIp='0.0.0.0/0',
            IpProtocol='TCP',
            FromPort=port,
            ToPort=port
        )
    except Exception as e:
        print(e)

In [20]:
def delete_aws_resources(iam_client, redshift_client):
    if not redshift_client and not iam_client:
        return
    
    redshift_client.delete_cluster(ClusterIdentifier=config.get('CLUSTER','CLUSTER_IDENTIFIER'), SkipFinalClusterSnapshot=True)
    iam_client.detach_role_policy(RoleName=config.get('IAM_ROLE','IAM_ROLE_NAME'), PolicyArn="arn:aws:iam::aws:policy/AmazonS3ReadOnlyAccess")
    iam_client.delete_role(RoleName=config.get('IAM_ROLE','IAM_ROLE_NAME'))

In [21]:
open_access_to_cluster(ec2_client, redshift_client)

ec2.SecurityGroup(id='sg-0133c01b457d4df9f')


In [1]:
redshift_cluster_status(redshift_client, verbose=True)

NameError: name 'redshift_cluster_status' is not defined

In [2]:
import configparser
import psycopg2
from sql_queries import create_table_queries, drop_table_queries



In [3]:
config = configparser.ConfigParser()
config.read('capstone.cfg')

conn = psycopg2.connect("host={} dbname={} user={} password={} port={}".format(
    config.get('CLUSTER','HOST'),
    config.get('CLUSTER','DB_NAME'),
    config.get('CLUSTER','DB_USER'),
    config.get('CLUSTER','DB_PASSWORD'),
    config.get('CLUSTER','DB_PORT')
))

OperationalError: could not connect to server: Connection timed out
	Is the server running on host "dwhcluster.cqgurtquub8h.us-west-2.redshift.amazonaws.com" (35.163.62.158) and accepting
	TCP/IP connections on port 5439?


In [11]:
cluster_props = redshift_cluster_status(redshift_client)

In [ ]:
if not cluster_props:
        return
    
    try:
        vpc = ec2_client.Vpc(id=cluster_props['VpcId'])
        defaultSg = list(vpc.security_groups.all())[0]
        print(defaultSg)
        port = int(config.get('CLUSTER','DB_PORT'))
        defaultSg.authorize_ingress(
            GroupName=defaultSg.group_name,
            CidrIp='0.0.0.0/0',
            IpProtocol='TCP',
            FromPort=port,
            ToPort=port
        )

In [13]:
vpc = ec2_client.Vpc(id=cluster_props['VpcId'])

In [14]:
vpc

ec2.Vpc(id='vpc-0720ee69b410d2f97')

In [17]:
defaultSg = list(vpc.security_groups.all())[-1]

In [18]:
port = int(config.get('CLUSTER','DB_PORT'))

In [19]:
port

5439

In [20]:
defaultSg.authorize_ingress(
    GroupName=defaultSg.group_name,
    CidrIp='0.0.0.0/0',
    IpProtocol='TCP',
    FromPort=port,
    ToPort=port
)

{'ResponseMetadata': {'RequestId': 'e88cace5-cadf-47aa-8754-54e349761fb4',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': 'e88cace5-cadf-47aa-8754-54e349761fb4',
   'cache-control': 'no-cache, no-store',
   'strict-transport-security': 'max-age=31536000; includeSubDomains',
   'content-type': 'text/xml;charset=UTF-8',
   'content-length': '723',
   'date': 'Fri, 15 Jul 2022 08:48:34 GMT',
   'server': 'AmazonEC2'},
  'RetryAttempts': 0}}